## Импорты

In [9]:
import pandas as pd
import sqlite3

## Подключаемся к БД и читаем схему таблицы

In [10]:
conn = sqlite3.connect("/content/checking-logs-RES_ex02.sqlite")

schema = pd.read_sql("PRAGMA table_info(test);", conn)
schema

,cid,name,type,notnull,dflt_value,pk
0,0,uid,TEXT,0,None,0
1,1,labname,TEXT,0,None,0
2,2,first_commit_ts,TIMESTAMP,0,None,0
3,3,first_view_ts,TIMESTAMP,0,None,0


In [11]:
pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';", conn)

,name
0,pageviews
1,checker
2,deadlines
3,datamart
4,test
5,control


## Результаты для группы test

In [12]:
query_test = '''
WITH valid_users AS (
    SELECT uid
    FROM test
    WHERE labname != 'project1'
    GROUP BY uid
    HAVING
        SUM(CASE WHEN first_commit_ts < first_view_ts THEN 1 ELSE 0 END) > 0 AND
        SUM(CASE WHEN first_commit_ts > first_view_ts THEN 1 ELSE 0 END) > 0
)

SELECT
    CASE
        WHEN t.first_commit_ts < t.first_view_ts THEN 'before'
        ELSE 'after'
    END AS time,
    AVG((julianday(datetime(d.deadlines, 'unixepoch')) - julianday(t.first_commit_ts)) * 24) AS avg_diff
FROM test t
JOIN deadlines d ON t.labname = d.labs
JOIN valid_users vu ON vu.uid = t.uid
WHERE t.labname != 'project1'
GROUP BY time
'''
test_results = pd.read_sql(query_test, conn)
test_results

,time,avg_diff
0,after,105.229101
1,before,61.156438


## Результаты для группы control

In [13]:
query_control = '''
WITH valid_control_users AS (
    SELECT uid
    FROM control
    WHERE labname != 'project1'
    GROUP BY uid
    HAVING
        SUM(CASE WHEN first_commit_ts < first_view_ts THEN 1 ELSE 0 END) > 0 AND
        SUM(CASE WHEN first_commit_ts > first_view_ts THEN 1 ELSE 0 END) > 0
)

SELECT
    CASE
        WHEN c.first_commit_ts < c.first_view_ts THEN 'before'
        ELSE 'after'
    END AS time,
    AVG((julianday(datetime(d.deadlines, 'unixepoch')) - julianday(c.first_commit_ts)) * 24) AS avg_diff
FROM control c
JOIN deadlines d ON c.labname = d.labs
JOIN valid_control_users u ON u.uid = c.uid
WHERE c.labname != 'project1'
GROUP BY time
'''
control_results = pd.read_sql(query_control, conn)
control_results

,time,avg_diff
0,after,118.144425
1,before,99.901295


## Закрываем соединение с БД

In [14]:
conn.close()

## В тестовой группе люди начали сдавать проекты за 44 часа до дедлайна, в то время как в контроль группе этот показатель за то же время составлял 18 часов, что свидетельствует о том, что внедрение таблицы скорости сдачи действительно мотивирует студентов раньше сдавать проекты.